In [ ]:
# Problem Set #3 NYU
# Juan Martin Morelli
# N12699038

import requests
import matplotlib.pyplot as plt
import re

def price_quote(sym):
	r = requests.get('http://finance.yahoo.com/d/quotes.csv',{'s': sym, 'f': 'p2'})
	m=re.search(r'([+-]?\d+\.?\d*)%?',r.text)
	res=float(m.group(1)) if m else []
	return res

price = []
price_abs = []
mkt_cap2 = []

data_file = open('company_list_corrected.csv','r')
data_file.next()

i=0
for line in data_file:
	line = line.replace(', ',' ')
	line = line.replace(',",','",')
	line = line.replace(',Inc',' Inc')
	line = line.replace('"','')
	sym, name, mkt_cap = line.split(',')
	m = re.search(r'(\d+\.?\d*)([MB]?)',mkt_cap)


	 # Need to put all mkt caps in the same units.
     # I will use millions of USD since is intermediate unit

	if m:
		if m.group(2)=='':
			mkt_cap2.append(float(m.group(1))) # assume they are already in millions
		elif m.group(2)=='M':
			mkt_cap2.append(float(m.group(1))) # leave them in millions
		elif m.group(2)=='B':
			mkt_cap2.append(float(m.group(1))*1000) # transform to millions
	else:
		mkt_cap2.append(0.0)

	price.append(price_quote(sym))
	i+=1



PA = [abs(price[i]) for i in range(len(price)) if mkt_cap2[i] > 0.0]
fig1, ax1 = plt.subplots()
plt.scatter(MA, PA , color = 'red')
ax1.set_title('Absolute Change - Market Cap')
ax1.set_xlabel('Mkt Cap (M)')
ax1.set_ylabel('Abs % Chg')
plt.show()

# At first sight it seems that the lower the market cap, the higher the range of the
# absolute change. That would suggest a higher variance the lower it is the market cap.

# To see it better, it is conviniente to take the logs of the market cap.
# So I "cheat" a bit and also import the math module to take logs
import math
lnMA = [math.log(mkt_cap2[i]) for i in range(len(mkt_cap2)) if mkt_cap2[i] > 0.0]
fig2, ax2 = plt.subplots()
plt.scatter(MA, PA , color = 'red')
ax2.set_title('Absolute Change - ln(Market Cap)')
ax2.set_xlabel('Log of Mkt Cap (M)')
ax2.set_ylabel('Abs % Chg')
plt.show()

# With a bit of imagination and good will, we can see a humped shaped plot,
# the maximum being nearer the lowest market cap firms.
# Once again, this suggests a negative relationship between market cap and
# daily return variance.